# Practical Exam: Spectrum Shades LLC
Spectrum Shades LLC is a prominent supplier of concrete color solutions, offering a wide range of pigments and coloring systems used in various concrete applications, including decorative concrete, precast concrete, and concrete pavers. The company prides itself on delivering high-quality colorants that meet the unique needs of its diverse clientele, including contractors, architects, and construction companies.
</br></br>
The company has recently observed a growing number of customer complaints regarding inconsistent color quality in their products. The discrepancies have led to a decline in customer satisfaction and a potential increase in product returns.
By identifying and mitigating the factors causing color variations, the company can enhance product reliability, reduce customer complaints, and minimize return rates.
</br></br>
You are part of the data analysis team tasked with providing actionable insights to help Spectrum Shades LLC address the issues of inconsistent color quality and improve customer satisfaction.

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `production_data` table, and the data team have provided the following data description. 

Write a query to ensure the data matches the description provided, including identifying and cleaning all invalid values. You must match all column names and description criteria.
</br>

- You should start with the data in the file "production_data.csv".
- Your output should be a DataFrame named clean_data.
- All column names and values should match the table below.
</br>

| Column Name             | Criteria                                                                                         |
|--------------------------|--------------------------------------------------------------------------------------------------|
| batch_id | Discrete. Identifier for each batch. Missing values are not possible. |
| production_date | Date. Date when the batch was produced.|
| raw_material_supplier | Categorical. Supplier of the raw materials. (1='national_supplier', 2='international_supplier'). <br> Missing values should be replaced with 'national_supplier'.|
| pigment_type           | Nominal. Type of pigment used. ['type_a', 'type_b', 'type_c']. <br> Missing values should be replaced with 'other'. |
| pigment_quantity       | Continuous. Amount of pigment added (in kilograms) (Range: 1 - 100). <br> Missing values should be replaced with median. |
| mixing_time           | Continuous. Duration of the mixing process (in minutes). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |
| mixing_speed          | Categorical. Speed of the mixing process represented as categories: 'Low', 'Medium', 'High'.</br> Missing values should be replaced with 'Not Specified'. |
| product_quality_score | Continuous. Overall quality score of the final product (rating on a scale of 1 to 10). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |


In [8]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('production_data.csv')

# --- Data Cleaning ---
# 1. batch_id: drop rows with missing batch_id
clean = df.dropna(subset=['batch_id'])

# 2. production_date: ensure valid date
clean['production_date'] = pd.to_datetime(clean['production_date'], errors='coerce')
clean = clean.dropna(subset=['production_date'])

# 3. raw_material_supplier: 1='national_supplier', 2='international_supplier', missing->'national_supplier'
clean['raw_material_supplier'] = clean['raw_material_supplier'].replace({1: 'national_supplier', 2: 'international_supplier'})
clean['raw_material_supplier'] = clean['raw_material_supplier'].fillna('national_supplier')

# 4. pigment_type: only ['type_a', 'type_b', 'type_c'], fix case, missing->'other'
clean['pigment_type'] = clean['pigment_type'].str.lower().replace({'type_a':'type_a','type_b':'type_b','type_c':'type_c'})
clean['pigment_type'] = clean['pigment_type'].where(clean['pigment_type'].isin(['type_a','type_b','type_c']), 'other')
clean['pigment_type'] = clean['pigment_type'].fillna('other')

# 5. pigment_quantity: 1-100, missing->median, out of range->nan then fill
clean['pigment_quantity'] = pd.to_numeric(clean['pigment_quantity'], errors='coerce')
clean.loc[(clean['pigment_quantity'] < 1) | (clean['pigment_quantity'] > 100), 'pigment_quantity'] = np.nan
pigment_quantity_median = clean['pigment_quantity'].median()
clean['pigment_quantity'] = clean['pigment_quantity'].fillna(pigment_quantity_median)

# 6. mixing_time: missing->mean (rounded 2 decimals)
clean['mixing_time'] = pd.to_numeric(clean['mixing_time'], errors='coerce')
mixing_time_mean = round(clean['mixing_time'].mean(), 2)
clean['mixing_time'] = clean['mixing_time'].fillna(mixing_time_mean)

# 7. mixing_speed: only 'Low', 'Medium', 'High', missing/invalid->'Not Specified'
valid_speeds = ['Low', 'Medium', 'High']
clean['mixing_speed'] = clean['mixing_speed'].where(clean['mixing_speed'].isin(valid_speeds), 'Not Specified')
clean['mixing_speed'] = clean['mixing_speed'].fillna('Not Specified')

# 8. product_quality_score: 1-10, missing->mean (rounded 2 decimals), out of range->nan then fill
clean['product_quality_score'] = pd.to_numeric(clean['product_quality_score'], errors='coerce')
clean.loc[(clean['product_quality_score'] < 1) | (clean['product_quality_score'] > 10), 'product_quality_score'] = np.nan
pqs_mean = round(clean['product_quality_score'].mean(), 2)
clean['product_quality_score'] = clean['product_quality_score'].fillna(pqs_mean)

# Final DataFrame for Task 1
clean_data = clean.copy()

# Task 2

You want to understand how the supplier type and quantity of materials affect the final product attributes.

Calculate the average `product_quality_score` and `pigment_quantity` grouped by `raw_material_supplier`.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named aggregated_data.
- It should include the three columns: `raw_material_supplier`, `avg_product_quality_score`, and `avg_pigment_quantity`.
- Your answers should be rounded to 2 decimal places.


In [2]:
# Group by raw_material_supplier, calculate means
agg = clean_data.groupby('raw_material_supplier').agg(
    avg_product_quality_score = ('product_quality_score', lambda x: round(x.mean(),2)),
    avg_pigment_quantity = ('pigment_quantity', lambda x: round(x.mean(),2))
).reset_index()
aggregated_data = agg

# Task 3

To get more insight into the factors behind product quality, you want to filter the data to see an average product quality score for a specified set of results.

Identify the average `product_quality_score` for batches with a `raw_material_supplier` of 2 and a `pigment_quantity` greater than 35 kg.

Write a query to return the average `avg_product_quality_score` for these filtered batches. Use the original production data table, not the output of Task 2.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named pigment_data.
- It should consist of a 1-row DataFrame with 3 columns: `raw_material_supplier`, `pigment_quantity`, and `avg_product_quality_score`.
- Your answers should be rounded to 2 decimal places where appropriate.


In [3]:
# Use original data, filter supplier==2 and pigment_quantity>35
pigment_df = df.copy()
pigment_df = pigment_df[pigment_df['raw_material_supplier'] == 2]
pigment_df = pigment_df[pigment_df['pigment_quantity'] > 35]

# Calculate means (after filtering)
if not pigment_df.empty:
    avg_pqs = round(pigment_df['product_quality_score'].mean(),2)
    avg_pq = round(pigment_df['pigment_quantity'].mean(),2)
    pigment_data = pd.DataFrame({
        'raw_material_supplier': [2],
        'pigment_quantity': [avg_pq],
        'avg_product_quality_score': [avg_pqs]
    })
else:
    pigment_data = pd.DataFrame({
        'raw_material_supplier': [2],
        'pigment_quantity': [np.nan],
        'avg_product_quality_score': [np.nan]
    })

# Task 4

In order to proceed with further analysis later, you need to analyze how various factors relate to product quality. Start by calculating the mean and standard deviation for the following columns: `pigment_quantity`, and `product_quality_score`. </br> These statistics will help in understanding the central tendency and variability of the data related to product quality.
</br> </br >
Next, calculate the Pearson correlation coefficient between the following variables: `pigment_quantity`, and `product_quality_score`.
</br>
These correlation coefficients will provide insights into the strength and direction of the relationships between the factors and overall product quality.


- You should start with the data in the file 'production_data.csv'.
- Calculate the mean and standard deviation for the columns pigment_quantity and product_quality_score as: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`.
- Calculate the Pearson correlation coefficient between pigment_quantity and product_quality_score as: `corr_coef`
- Your output should be a DataFrame named product_quality.
- It should include the columns: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`, `corr_coef`.
- Ensure that your answers are rounded to 2 decimal places.


In [4]:
# Use clean_data for stats
pq_mean = round(clean_data['product_quality_score'].mean(),2)
pq_sd = round(clean_data['product_quality_score'].std(),2)
pig_mean = round(clean_data['pigment_quantity'].mean(),2)
pig_sd = round(clean_data['pigment_quantity'].std(),2)
corr = round(clean_data['pigment_quantity'].corr(clean_data['product_quality_score']),2)
product_quality = pd.DataFrame({
    'product_quality_score_mean': [pq_mean],
    'product_quality_score_sd': [pq_sd],
    'pigment_quantity_mean': [pig_mean],
    'pigment_quantity_sd': [pig_sd],
    'corr_coef': [corr]
})

# FORMATTING AND NAMING CHECK
Use the code block below to check that your outputs are correctly named and formatted before you submit your project.

This code checks whether you have met our automarking requirements: that the specified DataFrames exist and contain the required columns. It then prints a table showing ✅ for each column that exists and ❌ for any that are missing, or if the DataFrame itself isn't available.

If a DataFrame or a column in a DataFrame doesn't exist, carefully check your code again.

IMPORTANT: even if your code passes the check below, this does not mean that your entire submission is correct. This is a check for naming and formatting only.

In [ ]:
import pandas as pd

def check_columns(output_df, output_df_name, required_columns):
    results = []
    for col in required_columns:
        exists = col in output_df.columns
        results.append({'Dataset': output_df_name, 'Column': col, 'Exists': '✅' if exists else '❌'})
    return results

def safe_check(output_df_name, required_columns):
    results = []
    if output_df_name in globals():
        obj = globals()[output_df_name]
        if isinstance(obj, pd.DataFrame):
            results.extend(check_columns(obj, output_df_name, required_columns))
        elif isinstance(obj, str) and ("SELECT" in obj.upper() or "FROM" in obj.upper()):
            results.append({'Dataset': output_df_name, 'Column': '—', 'Exists': 'ℹ️ SQL query string'})
        else:
            results.append({'Dataset': output_df_name, 'Column': '—', 'Exists': '❌ Not a DataFrame or query'})
    else:
        results.append({'Dataset': output_df_name, 'Column': '—', 'Exists': '❌ Variable not defined'})
    return results

requirements = {
    'clean_data': ['production_date', 'pigment_type', 'mixing_time', 'mixing_speed'],
    'aggregated_data': ['raw_material_supplier', 'avg_product_quality_score', 'avg_pigment_quantity'],
    'pigment_data': ['raw_material_supplier', 'pigment_quantity', 'avg_product_quality_score'],
    'product_quality': ['product_quality_score_mean', 'product_quality_score_sd',
                        'pigment_quantity_mean', 'pigment_quantity_sd', 'corr_coef']
}

all_results = []
for output_df_name, cols in requirements.items():
    all_results += safe_check(output_df_name, cols)

check_results_df = pd.DataFrame(all_results)

print(check_results_df)